In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import date
from sklearn.metrics import mean_squared_error

In [2]:
# THIS IS PURELY FOR TEST PURPOSE NEVER DO THAT IN PRODUCTION
POSTGRES_ADDRESS='postgres'
POSTGRES_PORT='5432'
POSTGRES_USERNAME='postgres'
POSTGRES_PASSWORD='postgres'

postgres_conn = 'postgresql://{0}:{1}@{2}:{3}/'.format(POSTGRES_USERNAME, POSTGRES_PASSWORD, POSTGRES_ADDRESS, POSTGRES_PORT)

# Create the connection
cnx = create_engine(postgres_conn)


Read in the Avocado Prices csv file as a DataFrame called df

In [3]:
# Parameterized

filepath='avocado.csv'
ml_id='auto_100'
ml_date=date.today()
n_estimators = 100
max_features = 'auto'

In [4]:
# Parameters
filepath = "/tmp/avocado.csv"
n_estimators = 150
max_features = "auto"
ml_id = "auto_150"


In [5]:
df= pd.read_csv(filepath)

The Feature "Unnamed:0" is just a representation of the indexes, so it's useless to keep it, lets remove it !

In [6]:
df.drop('Unnamed: 0',axis=1,inplace=True)

well as a first observation we can see that we are lucky, we dont have any missing values (18249 complete data) and 13 columns.
Now let's do some Feature Engineering on the Date Feature so we can be able to use the day and the month columns in building our machine learning model later. ( I didn't mention the year because its already there in data frame)

In [7]:
df['Date']=pd.to_datetime(df['Date'])
df['Month']=df['Date'].apply(lambda x:x.month)
df['Day']=df['Date'].apply(lambda x:x.day)

as we can see we have 54 regions and 2 unique types, so it's going to be easy to to transform the type feature to dummies, but for the region its going to be a bit complexe so I decided to drop the entire column.
I will drop the Date Feature as well because I already have 3 other columns for the Year, Month and Day.

In [8]:
df_final=pd.get_dummies(df.drop(['region','Date'],axis=1),drop_first=True)

Now our data are ready! lets apply our model which is going to be the Linear Regression because our Target variable 'AveragePrice'is continuous.
Let's now begin to train out regression model! We will need to first split up our data into an X array that contains the features to train on, and a y array with the target variable

In [9]:
X=df_final.iloc[:,1:14]
y=df_final['AveragePrice']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

Creating and Training the Model

In [10]:
from sklearn.ensemble import RandomForestRegressor
rdr = RandomForestRegressor(
    n_estimators=n_estimators,
    max_features=max_features
)
rdr.fit(X_train,y_train)
pred=rdr.predict(X_test)

In [11]:
#%%capture cap --no-stderr
#print(np.sqrt(mean_squared_error(y_test, pred)))

In [12]:
#with open(output_rmse, 'w') as f:
#    f.write(cap.stdout)

In [13]:
cnx.execute("""
    INSERT INTO ml.accuracies (ml_id, ml_date, accuracy) VALUES ('{0}', '{1}', {2}) 
    ON CONFLICT (ml_id, ml_date) 
    DO
        UPDATE SET accuracy = {2};
    """.format(ml_id, ml_date, np.sqrt(mean_squared_error(y_test, pred))))